# Bose Hubbard Hamiltonian Generator PYTHON 3

This code will be used to generate the Hubbard Hamiltonian. In this case, we are dealing only with the Bose-Hubbard Hamiltonian. **We assume the bosons have spin = 0.**

This notebook is specifically for my Github Page.

In [1]:
import math as m
import numpy as np
from matplotlib import pyplot as plt
from numpy import linalg as la
from pprint import pprint


## Matrix Elements

I will break down the Hamiltonian into the following:

$$ \begin{equation} 
H = H(J) + H(U)
\end{equation} $$

Where, in order, we have the hopping term and onsite potential term.

In terms of operators, the elements take the following form:

$$\begin{align}
H(J) &=  -J \sum_{\langle i,j\rangle} a_i^\dagger a_j \\
H(U) &= \frac{U}{2} \sum_i n_i[n_i-1]\\
\end{align}$$



In [2]:
def factorial(N):
    #we need this to count basis states, etc.
    i = 1
    result = 1
    while i<=N:
        result = result*i
        i+=1
    return result

# This function fails if our occupation exceeds 9.
def string_to_array(state_string):
    # takes a string and returns an np.array([])
    temp = np.zeros(len(state_string), dtype = int) #initializing a 0 array with the lenght of the string
    i = 0
    while i < len(state_string):
        temp[i] = state_string[i]
        i+=1
    return temp


def array_to_string(state_array):
    #takes an np.array([]) and returns a string
    temp = ''
    i=0
    while i<np.size(state_array):
        temp = temp + str(state_array[i])
        i += 1
    return temp


def get_key(state, states_dict):
    #pass a state (string) and the dictionary of the states. Returns the key of the state if it is a valid state.
    if state == ''.zfill(len(state)): #checking to see if the result is a string of 0s, implying Hop failed. 
        #print("failed hop")
        return -1
    return list(states_dict.keys())[list(states_dict.values()).index(state)]

def normalize(vector):
    #returns a normalized vector. Use this for get_GS
    return vector / la.norm(vector)

def get_GS_vector(hamiltonian):
    e, vecs = np.linalg.eigh(hamiltonian) 
    inds = np.argsort(e) #sorting the eigenvalues and eigenvectors
    e = e[inds]
    vecs = vecs[:, inds]
    GS = normalize(vecs[:,0]) #first column of the matrix of eigenvalues
    return GS

def get_GS_dict(hamiltonian, basis_states):
    #returns the GS as a dict. The keys are the states in Fock space (i.e. 20, 11, 02) and the entries are the coefficients
    e, vecs = np.linalg.eigh(hamiltonian) 
    inds = np.argsort(e) #sorting the eigenvalues and eigenvectors
    e = e[inds]
    vecs = vecs[:, inds]
    GS = normalize(vecs[:,0]) #first column of the matrix of eigenvalues
    GS_state = dict()
    i = 0
    while i < np.size(GS):
        GS_state[basis_states[i]] = GS[i]
        i +=1
    return GS_state
    

In [14]:
# We now generate some necessary functions
#These are all the functions necessary for getting the Hamiltonian, and also the ground state

def apply_hop(index1, index2, state_string):
    #WE ASSUME INDEX1 != INDEX2
    #the operator takes the form c^+ 2 c1, so index 1 is the state we are destroying, and c2 is the state we are creating
    #the state we pass in is a string
    #it returns the coefficient in front, and then the resulting string
    
    state = string_to_array(state_string)
    coeff = 1

    if state[index2] == 0:
        string = array_to_string(np.zeros(np.size(state), dtype = int))
        return coeff, string
        
    else:
        newstate = state.copy()
        #we count negatives by counting how many times c has to commute with elements
        #recall that [c_i, c+_j] = delta_ij = c_i c+_j - c+_j c_i. nonmatching indices commute
        #i.e, c_2|1,1>  = c_2 c+1 c+2|0> = c+1 (c_2 c+2) |0> = c+1 (1 + n_2) |0>
        #|3,0,0> = c+1 c+1 c+1 |0> /sqrt(6)
        #since [c_i, c+_j] = delta_ij, we don't really need to worry about commutation, negative signs, etc.
        #the coefficient will just be n_site - 1 on the destruction site.
        
        #find the coefficient that the operator yields
        coeff = m.sqrt(newstate[index2])
        
        #decrease the occupation number, and increase the other occupation number
        newstate[index2] = newstate[index2] - 1
        newstate[index1] = newstate[index1] + 1
        #for index1, we find the coefficient afterward because it is sqrt(n+1)
        coeff = coeff * m.sqrt(newstate[index1])
        newstate_string = array_to_string(newstate)
        return coeff, newstate_string

    
def get_Hop_Matrix(numsites, numparticles, basis_states, PBC):
    num_states = int(factorial(numsites - 1 +numparticles)/(factorial(numsites-1)*factorial(numparticles)))
    HJ = np.zeros([num_states, num_states])
    
    #list of strings of all posible index1 index2. i.e. c+1 c2 = "12"
    hop_index_set = set()
    site_list = ''
    number = 0
    
    while number < numsites:
        site_list = site_list + str(number)
        number+=1
    c = 0
    if(PBC):
        limit = numsites
    else:
        limit = numsites - 1
        
    c = 0    
    while c < limit:
        string = site_list[c] + site_list[(c+1)%numsites]
        hop_index_set.add(string)
        
        string = site_list[(c+1)%numsites] + site_list[c]
        hop_index_set.add(string)
        c += 1
    
    for key in basis_states.keys():
        for indices in hop_index_set:
            coefficient, state_string = apply_hop(int(indices[0]), int(indices[1]), basis_states[key])
            hopped_state = get_key(state_string, basis_states)
            temp_array=np.zeros(num_states, dtype = int)
            if hopped_state != -1:
                temp_array[hopped_state] = 1 
                #the resulting vector should be a column vector with just one 1 in the array
                HJ[:,key] = HJ[:,key] + coefficient * temp_array
    return -1*HJ #it's -J in the front, so multiple by -1


def get_U_Matrix(numsites, numparticles, basis_states):
    #returns the matrix H(U), but does NOT multiple by U
    #we will operate n_i(n_i-1) on all the sides
    # U matrix doesnt care if we have periodic boundary conditions or not
    
    num_states = int(factorial(numsites - 1 +numparticles)/(factorial(numsites-1)*factorial(numparticles)))
    U_matrix = np.zeros([num_states, num_states])
    
    for key in basis_states.keys():
        current_state = basis_states[key]
        i = 0
        coeff = 0
        while i < len(current_state):
            number_op = int(current_state[i]) 
            #print("Current number op is: " + str(number_op))
            coeff = coeff + number_op * (number_op - 1)
            i += 1
        coeff = coeff * 0.5
        #print("The coefficient is: " + str(coeff))
        #state_index = get_key(current_state, basis_states)
        #^ This is exacly 'key'
        temp_array=np.zeros(num_states, dtype = int)
        temp_array[key] = 1
        U_matrix[:,int(key)]= U_matrix[:,key] +  coeff*temp_array
        #print(U_matrix)
    return U_matrix  


def get_Hamiltonian(numsites, numparticles, basis_states, U, PBC = True):
    return (U * get_U_Matrix(numsites, numparticles, basis_states) + get_Hop_Matrix(numsites, numparticles, basis_states, PBC))


basis_states_dict_32 = {0: '200', 1: '020', 2: '002', 3: '110',4:'101', 5:'011'}     
basis_states_dict_22 = {0: '20', 1:'11', 2:'02'}
basis_states_dict_33 = {0: '300', 1: '030', 2: '003', 3: '210',4:'201', 5:'120', 6: '021', 7: '102', 8: '012', 9:'111'}     

# Correlation Function Functions

This block contains code for various correlation functions, and other necessary functions that could be helpful.

We define Von Neumann entropy as the following:

$$ S = -Tr( \rho ln(\rho)) $$

where $\rho$ is the denity matrix.

In [60]:
def create_density_matrix_from_vector(vector):
    return np.outer(vector, vector)

def VonNeumann_Entropy(p):
    evals = np.linalg.eigvals(p)
    for i in range(0, np.size(evals)):
        if abs(evals[i]) < 1e-15:
            evals[i] = 0.0
            
    log_base_p = np.diag(evals)
    S = - np.trace(log_base_p @ (np.log(log_base_p)))
    return(S)

def entropy_correlation(GS): # This function is a way to call the above easily taking in the ground state
    GS_density = create_density_matrix_from_vector(GS)
    return VonNeumann_Entropy(GS_density)


def VN_Entanglement_Entropy(GS_vec, basis_states, num_sites, num_particles):
    # takes in the ground state as a vector and the basis_states dictionary.
    # Also takes in num_sites. We need this for the entanglement entropy
    # num_particles is the number of particles
    
    reduced_coefficient = np.zeros(num_particles + 1) # we used this to store the reduced density matrix
    # if we have 2 particles, we need 0, 1, 2, so we add 1
    for i in range(0, np.size(GS_vec) - 1):
        basis_element = basis_states[i] # this will iterate over all states. This is a string
        
        index = int(basis_element[0])
        reduced_coefficient[index] = reduced_coefficient[index] + GS_vec[i]*np.conj(GS_vec[i]) #in the density matrix it would be squared
    S = 0
    for element in reduced_coefficient:
        if (element ==0):
            pass
        else:
            S = S - element * np.log(element)   
    return S